In [68]:
from pyscf import gto, lib
import numpy as np

np.set_printoptions(16, threshold=100000000, suppress=False, linewidth=300)

In [69]:
basis = {
    "C": "cc-pVTZ",
    "H": "cc-pVTZ",
    "Sb": "cc-pVTZ-PP",
}

basis_ecp = {
    "Sb": "cc-pVTZ-PP",
}

mol = gto.Mole(atom="""
Sb        -1.33937843      0.44597852     -1.27279684
Sb         1.33937843     -0.44597852     -1.27279684
C         -1.40429524      1.10441871      0.83468205
C         -2.16210130     -1.56132398     -0.84717555
C          2.16210130      1.56132398     -0.84717555
C          1.40429524     -1.10441871      0.83468205
H         -0.69918639      1.91987631      1.00872018
H         -1.16111477      0.29030616      1.51873028
H         -2.40124532      1.47235562      1.08516843
H         -2.02002046     -2.22909286     -1.69887295
H         -1.69052287     -2.01612927      0.02577778
H         -3.23450854     -1.49489801     -0.65423339
H          2.02002046      2.22909286     -1.69887295
H          3.23450854      1.49489801     -0.65423339
H          1.69052287      2.01612927      0.02577778
H          0.69918639     -1.91987631      1.00872018
H          2.40124532     -1.47235562      1.08516843
H          1.16111477     -0.29030616      1.51873028
""", basis=basis, ecp=basis_ecp).build()

## cint_prop

In [71]:
mol.has_ecp()

True

In [72]:
mol.nbas

130

In [76]:
mol.atom_charge(0)

23

In [73]:
mol.nao_nr()

366

In [74]:
mol.nao_cart()

410

In [75]:
mol.nao_2c()

732